<a href="https://colab.research.google.com/github/divx1979/NLP_Sub/blob/master/PROJECT_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('omw-1.4')
import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# import chardet
# with open('/content/spam.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(100000))
# result

In [5]:
data = pd.read_csv('/content/SMSSpamCollection', sep='\t', names = ['labels', 'messages'])

In [6]:
data.head()

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords

corpus = []
lmt = WordNetLemmatizer()

In [8]:
for i in range(len(data)):
  rev = re.sub('[^a-zA-Z]', ' ', data['messages'][i])
  rev = rev.lower()
  rev = rev.split()
  rev = [lmt.lemmatize(word) for word in rev if not word in stopwords.words('english')]
  rev = ' '.join(rev)
  corpus.append(rev)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features= 2500)
X = tf.fit_transform(corpus).toarray()

In [16]:
X.shape

(5572, 2500)

In [17]:
y = pd.get_dummies(data['labels'])

In [18]:
y.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [19]:
# Now here, we need only 1 col(either ham or spam) as you can see above
# if ham = 0, then automatically, its a spam = 1

y = y.iloc[:, 1].values

In [21]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [31]:
## now that we have both Independent Features(X) as well as the Dependent Features(y)
# we can split the model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 0)

In [32]:
X_test.shape

(1672, 2500)

In [33]:
X_train.shape

(3900, 2500)

In [39]:
## now that we have divided the data into train and test, we need to use a classification algorithm to solve this problem
# Naive Bayes is the BEST for NLP problemsin order to classify them

from sklearn.naive_bayes import MultinomialNB

In [40]:
spam_model = MultinomialNB().fit(X_train, y_train)

In [41]:
y_pred = spam_model.predict(X_test)

In [42]:
y_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [43]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [44]:
cm = confusion_matrix(y_test, y_pred)

In [45]:
cm

array([[1451,    0],
       [  30,  191]])

In [46]:
ac_score = accuracy_score(y_test, y_pred)

In [47]:
ac_score

0.9820574162679426